# Elastic Rod Model

In [295]:
from IPython.display import display, Markdown
import random
def latexify(x):
    out = '$' + x + '$'
    return out

def lprint(x):
    display(Markdown(latexify(latex(x))))
    
%run -i 'implicit.py'

In this notebook a discretisation of the elastic rod bifurcation problem will be investigated

https://pubs.rsc.org/en/content/chapterhtml/2015/bk9781849738132-00001?isbn=978-1-84973-813-2&sercode=bk

derives a system of equations that correspond to a period truss network model for an elastic rod.

In [68]:
def get_funcs(n, k = 1, f = None, fix_end_y = True):
    # returns a set of equations to solve
    if f is None:
        var('x')
        f = x # default spring energy function
        
    var_dict = {}
    for i in range(0,n+1):
        # create a grid
        # Pij is the displacement from the point (i,j) on the grid
        # then split into x,y position of where that point goes
        var_dict['p{}0x'.format(i)] = var('p{}0x'.format(i))
        var_dict['p{}0y'.format(i)] = var('p{}0y'.format(i))
        var_dict['p{}1x'.format(i)] = var('p{}1x'.format(i))
        var_dict['p{}1y'.format(i)] = var('p{}1y'.format(i))
        
    # enforce fixed points at left side of truss network
    var_dict['p00x'] = 0
    var_dict['p00y'] = 0
    var_dict['p01x'] = 0
    var_dict['p01y'] = 0 
    
    if fix_end_y == True:
        var_dict['p{}0y'.format(n)] = 0
        var_dict['p{}1y'.format(n)] = 0 
    
    #var_dict['p{}0x'.format(n)] = 0 # really n
    #var_dict['p{}0y'.format(n)] = 0
    #var_dict['p{}1x'.format(n)] = 0 # really n
    #var_dict['p{}1y'.format(n)] = 0 # really 1
    
    var_dict['l'] = var('l', latex_name=r'\lambda') # lamdba force variable
    
    # so after constraints at endpoints we have 4(n-1) uknowns
    
    funcs = []
    e_truss = 0
    for j in range(1,n+1):
        # iterate through each square in the truss
        # get the length of each of the five springs
        
        top_x = 1 + var_dict['p{}1x'.format(j)] - var_dict['p{}1x'.format(j-1)]
        top_y = var_dict['p{}1y'.format(j)] - var_dict['p{}1y'.format(j-1)]  
        top = f(sqrt(top_x^2 + top_y^2))
        
        right_x = var_dict['p{}1x'.format(j)] - var_dict['p{}0x'.format(j)] 
        right_y = 1 + var_dict['p{}1y'.format(j)] - var_dict['p{}0y'.format(j)] 
        right = f(sqrt(right_x^2 + right_y^2))
 
        bottom_x = 1 + var_dict['p{}0x'.format(j)] - var_dict['p{}0x'.format(j-1)]
        bottom_y = var_dict['p{}0y'.format(j)] - var_dict['p{}0y'.format(j-1)]  
        bottom = f(sqrt(bottom_x^2 + bottom_y^2))
        
        
        # \ spring
        diag1_x = 1 + var_dict['p{}0x'.format(j)] - var_dict['p{}1x'.format(j-1)]
        diag1_y = 1 + var_dict['p{}1y'.format(j-1)] - var_dict['p{}0y'.format(j)]
        diag1 = f(sqrt(diag1_x^2 + diag1_y^2)/sqrt(2)) # scale by root 2
        
        
        # / spring
        diag2_x = 1 + var_dict['p{}1x'.format(j)] - var_dict['p{}0x'.format(j-1)]
        diag2_y = 1 + var_dict['p{}1y'.format(j)] - var_dict['p{}0y'.format(j-1)]
        diag2 = f(sqrt(diag1_x^2 + diag1_y^2)/sqrt(2)) # so that zero is indeed a solution 
        
        e_truss = e_truss + top + right + bottom + diag1 + diag2
        
        
    #print(e_truss)
    e_truss = e_truss -(var_dict['l'] * (var_dict['p{}1x'.format(n)] + var_dict['p{}0x'.format(n)])^2)
    # add work done by a force lambda
        
    # solutions will be local minima of the energy equation
    for j in range(1,n):
        # only care about 1,...,n
        # 0 and n are fixed points
        funcs.append(e_truss.diff(var_dict['p{}0x'.format(j)]))
        funcs.append(e_truss.diff(var_dict['p{}0y'.format(j)]))
        funcs.append(e_truss.diff(var_dict['p{}1x'.format(j)]))
        funcs.append(e_truss.diff(var_dict['p{}1y'.format(j)]))
        
    funcs.append(e_truss.diff(var_dict['p{}1x'.format(n)]))
    funcs.append(e_truss.diff(var_dict['p{}0x'.format(n)]))
    
    if fix_end_y is False:
        funcs.append(e_truss.diff(var_dict['p{}1y'.format(n)]))
        funcs.append(e_truss.diff(var_dict['p{}0y'.format(n)]))
        
    return funcs, var_dict
    

In [69]:
funcs, var_dict = get_funcs(2,f = (x-1)^2)

implicit.py:1: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  from math import factorial


In [70]:
var_dict

{'p00x': 0,
 'p00y': 0,
 'p01x': 0,
 'p01y': 0,
 'p10x': p10x,
 'p10y': p10y,
 'p11x': p11x,
 'p11y': p11y,
 'p20x': p20x,
 'p20y': 0,
 'p21x': p21x,
 'p21y': 0,
 'l': l}

In [71]:
lprint(funcs[5](0,0,0,0,0,0,0))

$ 0 $

In [72]:
len(funcs)

6

In [73]:
old_var = var_dict
var_dict = {key : var_dict[key] for key in var_dict.keys() if var_dict[key] != 0}
var_dict = {key : var_dict[key] for key in var_dict.keys() if key != 'l'}

J = jacobian(funcs,tuple(var_dict.values()))(**{var : 0 for var in var_dict.keys()})
lprint(J)

$ \left(\begin{array}{rrrrrr}
5 & -1 & 0 & 0 & -2 & 0 \\
-1 & 3 & 0 & -2 & 0 & 0 \\
0 & 0 & 5 & -1 & -1 & -2 \\
0 & -2 & -1 & 3 & 1 & 0 \\
0 & 0 & -2 & 0 & -2 \, {\lambda} & -2 \, {\lambda} + 2 \\
-2 & 0 & -1 & 1 & -2 \, {\lambda} + 3 & -2 \, {\lambda}
\end{array}\right) $

In [74]:
lprint(det(J))

$ 640 \, {\lambda} - 176 $

In [76]:
#bif_funcs = [func(l = 2/((32*sqrt(2)) + 36)) for func in funcs] #
#bif_funcs = [func(l = (sqrt(2)*(9/4))/((225*sqrt(2))/2 + 108)) for func in funcs] #
bif_funcs = [func(l = 176/640) for func in funcs]
J = jacobian(bif_funcs,tuple(var_dict.values()))(**{var : 0 for var in var_dict.keys()})
lprint(J)

$ \left(\begin{array}{rrrrrr}
5 & -1 & 0 & 0 & -2 & 0 \\
-1 & 3 & 0 & -2 & 0 & 0 \\
0 & 0 & 5 & -1 & -1 & -2 \\
0 & -2 & -1 & 3 & 1 & 0 \\
0 & 0 & -2 & 0 & -\frac{11}{20} & \frac{29}{20} \\
-2 & 0 & -1 & 1 & \frac{49}{20} & -\frac{11}{20}
\end{array}\right) $

### Orthogonalising the kernel

In [77]:
K = J.right_kernel().basis()
lprint(K)

$ \left[\left(1,\,\frac{3}{7},\,\frac{13}{7},\,\frac{1}{7},\,\frac{16}{7},\,\frac{24}{7}\right)\right] $

In [78]:
Kperp = matrix(K).right_kernel().basis()
lprint(Kperp)

$ \left[\left(1,\,0,\,0,\,0,\,0,\,-\frac{7}{24}\right), \left(0,\,1,\,0,\,0,\,0,\,-\frac{1}{8}\right), \left(0,\,0,\,1,\,0,\,0,\,-\frac{13}{24}\right), \left(0,\,0,\,0,\,1,\,0,\,-\frac{1}{24}\right), \left(0,\,0,\,0,\,0,\,1,\,-\frac{2}{3}\right)\right] $

In [79]:
A = matrix(K+Kperp).inverse().apply_map(lambda x: x.full_simplify())
lprint(A)

$ \left(\begin{array}{rrrrrr}
\frac{49}{1060} & \frac{1011}{1060} & -\frac{21}{1060} & -\frac{91}{1060} & -\frac{7}{1060} & -\frac{28}{265} \\
\frac{21}{1060} & -\frac{21}{1060} & \frac{1051}{1060} & -\frac{39}{1060} & -\frac{3}{1060} & -\frac{12}{265} \\
\frac{91}{1060} & -\frac{91}{1060} & -\frac{39}{1060} & \frac{891}{1060} & -\frac{13}{1060} & -\frac{52}{265} \\
\frac{7}{1060} & -\frac{7}{1060} & -\frac{3}{1060} & -\frac{13}{1060} & \frac{1059}{1060} & -\frac{4}{265} \\
\frac{28}{265} & -\frac{28}{265} & -\frac{12}{265} & -\frac{52}{265} & -\frac{4}{265} & \frac{201}{265} \\
\frac{42}{265} & -\frac{42}{265} & -\frac{18}{265} & -\frac{78}{265} & -\frac{6}{265} & -\frac{96}{265}
\end{array}\right) $

In [80]:
key_list = [str(var) for var in var_dict.values()]
value_list = list(A*vector(var_dict.values()))
mutate_dict = dict(zip(key_list, value_list))

In [169]:
mutate_dict

{'p10x': 49/1060*p10x + 1011/1060*p10y - 21/1060*p11x - 91/1060*p11y - 7/1060*p20x - 28/265*p21x,
 'p10y': 21/1060*p10x - 21/1060*p10y + 1051/1060*p11x - 39/1060*p11y - 3/1060*p20x - 12/265*p21x,
 'p11x': 91/1060*p10x - 91/1060*p10y - 39/1060*p11x + 891/1060*p11y - 13/1060*p20x - 52/265*p21x,
 'p11y': 7/1060*p10x - 7/1060*p10y - 3/1060*p11x - 13/1060*p11y + 1059/1060*p20x - 4/265*p21x,
 'p20x': 28/265*p10x - 28/265*p10y - 12/265*p11x - 52/265*p11y - 4/265*p20x + 201/265*p21x,
 'p21x': 42/265*p10x - 42/265*p10y - 18/265*p11x - 78/265*p11y - 6/265*p20x - 96/265*p21x}

In [81]:
Afuncs = [func(**mutate_dict) for func in bif_funcs]

In [82]:
lprint(Afuncs) 

$ \left[\frac{\sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}}{1060 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} - \frac{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{530 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{530 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} - \frac{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{265 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}}, \frac{\sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}}{1060 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} + \frac{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{530 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{530 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{{\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{265 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}}, -\frac{\sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}}{1060 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{530 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{530 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{265 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}}, \frac{\sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}}{1060 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{{\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{530 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{{\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{530 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{{\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{265 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}}, \frac{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{530 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{2 \, {\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225} - 265\right)}}{265 \, \sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225}} - \frac{77}{530} \, \mathit{p10x} + \frac{77}{530} \, \mathit{p10y} + \frac{33}{530} \, \mathit{p11x} + \frac{143}{530} \, \mathit{p11y} + \frac{11}{530} \, \mathit{p20x} - \frac{231}{1060} \, \mathit{p21x}, \frac{\sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}}{1060 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{530 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} - \frac{2 \, {\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225} - 265\right)}}{265 \, \sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225}} - \frac{77}{530} \, \mathit{p10x} + \frac{77}{530} \, \mathit{p10y} + \frac{33}{530} \, \mathit{p11x} + \frac{143}{530} \, \mathit{p11y} + \frac{11}{530} \, \mathit{p20x} - \frac{231}{1060} \, \mathit{p21x}\right] $

In [83]:
Ja = jacobian(Afuncs,tuple(var_dict.values()))(**{var : 0 for var in var_dict.keys()}).apply_map(lambda x: x.full_simplify())
lprint(Ja)

$ \left(\begin{array}{rrrrrr}
0 & 5 & -1 & 0 & 0 & -2 \\
0 & -1 & 3 & 0 & -2 & 0 \\
0 & 0 & 0 & 5 & -1 & -1 \\
0 & 0 & -2 & -1 & 3 & 1 \\
0 & 0 & 0 & -2 & 0 & -\frac{11}{20} \\
0 & -2 & 0 & -1 & 1 & \frac{49}{20}
\end{array}\right) $

In [84]:
Ka = Ja.right_kernel().basis()
lprint(Ka)

$ \left[\left(1,\,0,\,0,\,0,\,0,\,0\right)\right] $

### Orthogonalising the image

In [85]:
I = Ja.image().basis()
I = [Ja*vec for vec in I]
print(len(I))

5


In [86]:
Iperp = matrix(I).right_kernel().basis()
lprint(Iperp)

$ \left[\left(1,\,\frac{3}{7},\,\frac{13}{7},\,\frac{1}{7},\,\frac{24}{7},\,\frac{16}{7}\right)\right] $

In [87]:
B = matrix(I + Iperp).inverse().apply_map(lambda x: x.full_simplify())
lprint(B)

$ \left(\begin{array}{rrrrrr}
\frac{4921}{16960} & \frac{1469}{16960} & \frac{387}{16960} & -\frac{89}{16960} & \frac{87}{424} & \frac{49}{1060} \\
\frac{4207}{50880} & \frac{12441}{16960} & \frac{3829}{50880} & \frac{28657}{50880} & -\frac{191}{1272} & \frac{21}{1060} \\
-\frac{2263}{50880} & \frac{911}{16960} & \frac{8819}{50880} & \frac{6167}{50880} & -\frac{121}{1272} & \frac{91}{1060} \\
-\frac{357}{16960} & \frac{9447}{16960} & \frac{1721}{16960} & \frac{14373}{16960} & -\frac{139}{424} & \frac{7}{1060} \\
-\frac{817}{4240} & -\frac{293}{4240} & -\frac{539}{4240} & \frac{113}{4240} & -\frac{39}{106} & \frac{42}{265} \\
\frac{2341}{12720} & -\frac{637}{4240} & \frac{247}{12720} & -\frac{3749}{12720} & \frac{187}{318} & \frac{28}{265}
\end{array}\right) $

In [88]:
tfuncs = vector(Afuncs)*B

In [89]:
lprint(tfuncs)

$ \left(\frac{4921 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}}{17977600 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} + \frac{4207 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}}{53932800 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} + \frac{11627 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}}{53932800 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} - \frac{357 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}}{17977600 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} - \frac{2263 \, {\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{357 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{7541 \, {\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{357 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{5399 \, {\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{4207 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{4921 \, {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{4207 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} - \frac{8513 \, {\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{6741600 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} + \frac{2639 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{6741600 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} - \frac{599 \, {\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225} - 265\right)}}{210675 \, \sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225}} + \frac{847}{674160} \, \mathit{p10x} - \frac{847}{674160} \, \mathit{p10y} - \frac{121}{224720} \, \mathit{p11x} - \frac{1573}{674160} \, \mathit{p11y} - \frac{121}{674160} \, \mathit{p20x} + \frac{847}{449440} \, \mathit{p21x},\,\frac{1469 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}}{17977600 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} + \frac{12441 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}}{17977600 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} - \frac{3459 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}}{17977600 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{9447 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}}{17977600 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{911 \, {\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{9447 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{2083 \, {\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{9447 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{4017 \, {\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{12441 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{1469 \, {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{12441 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} - \frac{279 \, {\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{2247200 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} + \frac{1497 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{2247200 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} + \frac{43 \, {\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225} - 265\right)}}{70225 \, \sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225}} + \frac{7161}{224720} \, \mathit{p10x} - \frac{7161}{224720} \, \mathit{p10y} - \frac{3069}{224720} \, \mathit{p11x} - \frac{13299}{224720} \, \mathit{p11y} - \frac{1023}{224720} \, \mathit{p20x} + \frac{21483}{449440} \, \mathit{p21x},\,\frac{387 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}}{17977600 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} + \frac{3829 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}}{53932800 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} - \frac{7831 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}}{53932800 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{1721 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}}{17977600 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{8819 \, {\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{1721 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{15287 \, {\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{1721 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{173 \, {\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{3829 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{387 \, {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{3829 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{3829 \, {\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{6741600 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} - \frac{667 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{6741600 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} - \frac{233 \, {\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225} - 265\right)}}{210675 \, \sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225}} + \frac{10549}{674160} \, \mathit{p10x} - \frac{10549}{674160} \, \mathit{p10y} - \frac{1507}{224720} \, \mathit{p11x} - \frac{19591}{674160} \, \mathit{p11y} - \frac{1507}{674160} \, \mathit{p20x} + \frac{10549}{449440} \, \mathit{p21x},\,-\frac{89 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}}{17977600 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} + \frac{28657 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}}{53932800 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} - \frac{21163 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}}{53932800 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{14373 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}}{17977600 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{6167 \, {\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{14373 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{4811 \, {\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{14373 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{14729 \, {\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{28657 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} - \frac{89 \, {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{8988800 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{28657 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{26966400 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{3217 \, {\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{6741600 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} - \frac{7231 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{6741600 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} + \frac{511 \, {\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225} - 265\right)}}{210675 \, \sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225}} + \frac{26257}{674160} \, \mathit{p10x} - \frac{26257}{674160} \, \mathit{p10y} - \frac{3751}{224720} \, \mathit{p11x} - \frac{48763}{674160} \, \mathit{p11y} - \frac{3751}{674160} \, \mathit{p20x} + \frac{26257}{449440} \, \mathit{p21x},\,\frac{87 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}}{449440 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} - \frac{191 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}}{1348320 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} + \frac{869 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}}{1348320 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} - \frac{139 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}}{449440 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} - \frac{121 \, {\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{674160 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{139 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{224720 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{347 \, {\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{674160 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} - \frac{139 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{224720 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{487 \, {\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{674160 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} - \frac{191 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{674160 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{87 \, {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{224720 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} - \frac{191 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{674160 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} - \frac{191 \, {\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{168540 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} + \frac{113 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{168540 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} - \frac{304 \, {\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225} - 265\right)}}{42135 \, \sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225}} - \frac{539}{16854} \, \mathit{p10x} + \frac{539}{16854} \, \mathit{p10y} + \frac{77}{5618} \, \mathit{p11x} + \frac{1001}{16854} \, \mathit{p11y} + \frac{77}{16854} \, \mathit{p20x} - \frac{539}{11236} \, \mathit{p21x},\,\frac{49 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}}{1123600 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} + \frac{21 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}}{1123600 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x} - 1060\right)}^{2}}} + \frac{21 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}}{1123600 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{7 \, \sqrt{2} {\left(\sqrt{2} \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}} - 2120\right)} {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}}{1123600 \, \sqrt{{\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} - 9 \, \mathit{p11x} - 1099 \, \mathit{p11y} - 3 \, \mathit{p20x} + 1012 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x} + 1060\right)}^{2}}} + \frac{91 \, {\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{561800 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{7 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{561800 \, \sqrt{{\left(91 \, \mathit{p10x} - 91 \, \mathit{p10y} - 39 \, \mathit{p11x} + 891 \, \mathit{p11y} - 13 \, \mathit{p20x} - 208 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{77 \, {\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{561800 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{7 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{561800 \, \sqrt{{\left(77 \, \mathit{p10x} - 77 \, \mathit{p10y} - 33 \, \mathit{p11x} - 1203 \, \mathit{p11y} - 11 \, \mathit{p20x} - 176 \, \mathit{p21x} + 1060\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} - 3 \, \mathit{p11x} - 13 \, \mathit{p11y} + 1059 \, \mathit{p20x} - 16 \, \mathit{p21x}\right)}^{2}}} + \frac{63 \, {\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{561800 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{21 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{561800 \, \sqrt{{\left(63 \, \mathit{p10x} - 1123 \, \mathit{p10y} - 27 \, \mathit{p11x} - 117 \, \mathit{p11y} - 9 \, \mathit{p20x} + 916 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{49 \, {\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{561800 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{21 \, {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}} - 1060\right)}}{561800 \, \sqrt{{\left(49 \, \mathit{p10x} + 1011 \, \mathit{p10y} - 21 \, \mathit{p11x} - 91 \, \mathit{p11y} - 7 \, \mathit{p20x} - 112 \, \mathit{p21x} + 1060\right)}^{2} + {\left(21 \, \mathit{p10x} - 21 \, \mathit{p10y} + 1051 \, \mathit{p11x} - 39 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2}}} + \frac{21 \, {\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{140450 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} + \frac{7 \, {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)} {\left(\sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}} - 530\right)}}{140450 \, \sqrt{{\left(21 \, \mathit{p10x} - 551 \, \mathit{p10y} - 9 \, \mathit{p11x} + 491 \, \mathit{p11y} - 3 \, \mathit{p20x} - 48 \, \mathit{p21x}\right)}^{2} + {\left(7 \, \mathit{p10x} - 7 \, \mathit{p10y} + 527 \, \mathit{p11x} - 13 \, \mathit{p11y} - 531 \, \mathit{p20x} - 16 \, \mathit{p21x} - 530\right)}^{2}}} + \frac{28 \, {\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)} {\left(\sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225} - 265\right)}}{70225 \, \sqrt{{\left(14 \, \mathit{p10x} - 14 \, \mathit{p10y} - 6 \, \mathit{p11x} - 26 \, \mathit{p11y} - 2 \, \mathit{p20x} - 297 \, \mathit{p21x}\right)}^{2} + 70225}} - \frac{539}{14045} \, \mathit{p10x} + \frac{539}{14045} \, \mathit{p10y} + \frac{231}{14045} \, \mathit{p11x} + \frac{1001}{14045} \, \mathit{p11y} + \frac{77}{14045} \, \mathit{p20x} - \frac{1617}{28090} \, \mathit{p21x}\right) $

In [90]:
Jt = jacobian(tfuncs,tuple(var_dict.values()))(**{var : 0 for var in var_dict.keys()}).apply_map(lambda x: x.full_simplify())

In [91]:
lprint(Jt)

$ \left(\begin{array}{rrrrrr}
0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0
\end{array}\right) $

In [92]:
It = Jt.image().basis()
It = [Jt*vec for vec in It]
lprint(It)

$ \left[\left(1,\,0,\,0,\,0,\,0,\,0\right), \left(0,\,1,\,0,\,0,\,0,\,0\right), \left(0,\,0,\,1,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,1,\,0,\,0\right), \left(0,\,0,\,0,\,0,\,1,\,0\right)\right] $

In [93]:
var_dict_old = var_dict
var_dict_old

{'p10x': p10x,
 'p10y': p10y,
 'p11x': p11x,
 'p11y': p11y,
 'p20x': p20x,
 'p21x': p21x}

In [95]:
var_dict.values()

dict_values([p10x, p10y, p11x, p11y, p20x, p21x])

## Running the implicit function theorem

In [96]:
position = {key : 0 for key in var_dict_old.keys()}
position

{'p10x': 0, 'p10y': 0, 'p11x': 0, 'p11y': 0, 'p20x': 0, 'p21x': 0}

In [98]:
#code_funcs = [tfuncs[index].function(*list(var_dict_old.values())) for index in [0,1,2,3,4,5,6]] # careful with computer science indexing from zero
code_funcs = [tfuncs[index] for index in [0,1,2,3,4]] # the last function has degenerate first derivative always
position = {key : 0 for key in var_dict_old.keys()}
# var_dict already loaded
var_dict = {'y1' : p10y, 'y2' : p11x, 'y3' : p11y, 'y4' : p20x, 'y5' : p21x, 'x1' : p10x}
x_var_keys = ['p10x'] # the variables in the kernel
x_dim = 1
y_dim = 5 # invertible part

In [100]:
var_dict.keys()

dict_keys(['y1', 'y2', 'y3', 'y4', 'y5', 'x1'])

In [101]:
t_dict = TensorDict(code_funcs, position, var_dict, x_dim, y_dim)

computing Taylor approximaton to 3 order for speedup


In [146]:
out = get_hkx_polynomial(code_funcs, 3, x_dim, y_dim, var_dict, x_var_keys, t_dict, position)

100%|██████████| 1/1 [00:00<00:00, 342.76it/s]


[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]


100%|██████████| 4/4 [00:00<00:00, 24.46it/s]


[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]


100%|██████████| 14/14 [00:49<00:00,  3.51s/it]


[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]


In [147]:
out

[4196941012747/21277786683840000*p10x^3 - 330799/285843840*p10x^2,
 17581544619597/7092595561280000*p10x^3 - 1822261/476406400*p10x^2,
 8846863517401/21277786683840000*p10x^3 - 564137/1429219200*p10x^2,
 61563188467669/21277786683840000*p10x^3 - 5712077/1429219200*p10x^2,
 -428717692487/531944667096000*p10x^3 + 15631/35730480*p10x^2]

### Looking for more solutions

Now substitute this into the equation that had degenerate kernel

In [148]:
loc = [(value,0) for value in var_dict_old.values() ] + [3]
print(loc)

[(p10x, 0), (p10y, 0), (p11x, 0), (p11y, 0), (p20x, 0), (p21x, 0), 3]


In [149]:
#lprint(taylor(tfuncs[5], *loc))

In [150]:
tf_taylor = tfuncs[5].taylor(*[(key,value) for key,value in position.items()],3)
lprint(tf_taylor.polynomial(SR))

$ \left(-\frac{26411}{31561924000}\right) \mathit{p10x}^{3} + \left(-\frac{7858473}{63123848000}\right) \mathit{p10x}^{2} \mathit{p10y} + \frac{52671031}{31561924000} \mathit{p10x} \mathit{p10y}^{2} + \left(-\frac{3536489467}{63123848000}\right) \mathit{p10y}^{3} + \left(-\frac{37289931}{63123848000}\right) \mathit{p10x}^{2} \mathit{p11x} + \frac{99992193}{7890481000} \mathit{p10x} \mathit{p10y} \mathit{p11x} + \left(-\frac{5282188163}{63123848000}\right) \mathit{p10y}^{2} \mathit{p11x} + \left(-\frac{299890143}{31561924000}\right) \mathit{p10x} \mathit{p11x}^{2} + \frac{9982277907}{63123848000} \mathit{p10y} \mathit{p11x}^{2} + \frac{2660821723}{63123848000} \mathit{p11x}^{3} + \frac{9838269}{63123848000} \mathit{p10x}^{2} \mathit{p11y} + \left(-\frac{67557917}{7890481000}\right) \mathit{p10x} \mathit{p10y} \mathit{p11y} + \frac{7867873517}{63123848000} \mathit{p10y}^{2} \mathit{p11y} + \left(-\frac{33486453}{15780962000}\right) \mathit{p10x} \mathit{p11x} \mathit{p11y} + \frac{1178826887}{31561924000} \mathit{p10y} \mathit{p11x} \mathit{p11y} + \left(-\frac{3773377671}{63123848000}\right) \mathit{p11x}^{2} \mathit{p11y} + \frac{98308847}{31561924000} \mathit{p10x} \mathit{p11y}^{2} + \left(-\frac{6705192753}{63123848000}\right) \mathit{p10y} \mathit{p11y}^{2} + \left(-\frac{1389745091}{63123848000}\right) \mathit{p11x} \mathit{p11y}^{2} + \frac{2623462303}{63123848000} \mathit{p11y}^{3} + \left(-\frac{23482809}{63123848000}\right) \mathit{p10x}^{2} \mathit{p20x} + \left(-\frac{53753637}{15780962000}\right) \mathit{p10x} \mathit{p10y} \mathit{p20x} + \frac{1893138807}{63123848000} \mathit{p10y}^{2} \mathit{p20x} + \frac{6675123}{986310125} \mathit{p10x} \mathit{p11x} \mathit{p20x} + \left(-\frac{5211052959}{31561924000}\right) \mathit{p10y} \mathit{p11x} \mathit{p20x} + \left(-\frac{88522749}{63123848000}\right) \mathit{p11x}^{2} \mathit{p20x} + \frac{37057769}{7890481000} \mathit{p10x} \mathit{p11y} \mathit{p20x} + \left(-\frac{1533060179}{31561924000}\right) \mathit{p10y} \mathit{p11y} \mathit{p20x} + \frac{4579613451}{31561924000} \mathit{p11x} \mathit{p11y} \mathit{p20x} + \left(-\frac{607025209}{63123848000}\right) \mathit{p11y}^{2} \mathit{p20x} + \left(-\frac{879610711}{31561924000}\right) \mathit{p10x} \mathit{p20x}^{2} + \frac{6799077971}{63123848000} \mathit{p10y} \mathit{p20x}^{2} + \frac{696560193}{63123848000} \mathit{p11x} \mathit{p20x}^{2} + \left(-\frac{4370575447}{63123848000}\right) \mathit{p11y} \mathit{p20x}^{2} + \frac{772866241}{63123848000} \mathit{p20x}^{3} + \left(-\frac{36995637}{63123848000}\right) \mathit{p10x}^{2} \mathit{p21x} + \frac{104870829}{31561924000} \mathit{p10x} \mathit{p10y} \mathit{p21x} + \left(-\frac{377241221}{63123848000}\right) \mathit{p10y}^{2} \mathit{p21x} + \left(-\frac{196059927}{31561924000}\right) \mathit{p10x} \mathit{p11x} \mathit{p21x} + \frac{1846977279}{31561924000} \mathit{p10y} \mathit{p11x} \mathit{p21x} + \left(-\frac{5884503597}{63123848000}\right) \mathit{p11x}^{2} \mathit{p21x} + \frac{78548813}{31561924000} \mathit{p10x} \mathit{p11y} \mathit{p21x} + \frac{424613259}{31561924000} \mathit{p10y} \mathit{p11y} \mathit{p21x} + \frac{661916703}{31561924000} \mathit{p11x} \mathit{p11y} \mathit{p21x} + \left(-\frac{2259051557}{63123848000}\right) \mathit{p11y}^{2} \mathit{p21x} + \frac{63141057}{31561924000} \mathit{p10x} \mathit{p20x} \mathit{p21x} + \frac{229186167}{31561924000} \mathit{p10y} \mathit{p20x} \mathit{p21x} + \left(-\frac{449822373}{31561924000}\right) \mathit{p11x} \mathit{p20x} \mathit{p21x} + \left(-\frac{256702313}{31561924000}\right) \mathit{p11y} \mathit{p20x} \mathit{p21x} + \frac{4957060451}{63123848000} \mathit{p20x}^{2} \mathit{p21x} + \frac{308642033}{31561924000} \mathit{p10x} \mathit{p21x}^{2} + \left(-\frac{442843121}{31561924000}\right) \mathit{p10y} \mathit{p21x}^{2} + \left(-\frac{800876517}{31561924000}\right) \mathit{p11x} \mathit{p21x}^{2} + \frac{160983263}{31561924000} \mathit{p11y} \mathit{p21x}^{2} + \left(-\frac{167038893}{31561924000}\right) \mathit{p20x} \mathit{p21x}^{2} + \left(-\frac{1268233799}{15780962000}\right) \mathit{p21x}^{3} + \frac{1029}{5955080} \mathit{p10x}^{2} + \frac{168903}{29775400} \mathit{p10x} \mathit{p10y} + \frac{76559}{3721925} \mathit{p10y}^{2} + \frac{5537}{1191016} \mathit{p10x} \mathit{p11x} + \frac{1268799}{14887700} \mathit{p10y} \mathit{p11x} + \frac{229929}{2977540} \mathit{p11x}^{2} + \left(-\frac{94423}{29775400}\right) \mathit{p10x} \mathit{p11y} + \frac{137039}{7443850} \mathit{p10y} \mathit{p11y} + \left(-\frac{1292179}{14887700}\right) \mathit{p11x} \mathit{p11y} + \frac{39403}{7443850} \mathit{p11y}^{2} + \frac{208887}{29775400} \mathit{p10x} \mathit{p20x} + \left(-\frac{1297023}{14887700}\right) \mathit{p10y} \mathit{p20x} + \left(-\frac{13741}{3721925}\right) \mathit{p11x} \mathit{p20x} + \frac{1187823}{14887700} \mathit{p11y} \mathit{p20x} + \frac{2344587}{14887700} \mathit{p20x}^{2} + \frac{6419}{14887700} \mathit{p10x} \mathit{p21x} + \left(-\frac{211211}{14887700}\right) \mathit{p10y} \mathit{p21x} + \frac{423899}{14887700} \mathit{p11x} \mathit{p21x} + \left(-\frac{319109}{14887700}\right) \mathit{p11y} \mathit{p21x} + \left(-\frac{87591}{2977540}\right) \mathit{p20x} \mathit{p21x} + \frac{84259}{7443850} \mathit{p21x}^{2} $

In [167]:
var_dict_old

{'p10x': p10x,
 'p10y': p10y,
 'p11x': p11x,
 'p11y': p11y,
 'p20x': p20x,
 'p21x': p21x}

In [152]:
tf_sub = tf_taylor(p10y = out[0], p11x = out[1], p11y = out[2], p20x = out[3], p21x = out[4])

In [168]:
#local = sum([a[0]*a[1] for a in tf_sub.polynomial(SR) if a[1].degree() <= 2])
local = tf_sub.polynomial(SR).truncate(4)
lprint(local)

$ -\frac{2869269431}{55410902822500} \mathit{p10x}^{3} + \frac{1029}{5955080} \mathit{p10x}^{2} $

We see that this will have solutions, therefore the point $\lambda$ is indeed a bifurcation point, and the kernel dimension spans the solutions

### Generating more solutions

We have solved $f \circ A$ where $A$ orthogonalised the kernel, thus by applying $A$ we can get a traversel of the new solutions.

In [361]:
sols = [p10x] + out
sols

[p10x,
 4196941012747/21277786683840000*p10x^3 - 330799/285843840*p10x^2,
 17581544619597/7092595561280000*p10x^3 - 1822261/476406400*p10x^2,
 8846863517401/21277786683840000*p10x^3 - 564137/1429219200*p10x^2,
 61563188467669/21277786683840000*p10x^3 - 5712077/1429219200*p10x^2,
 -428717692487/531944667096000*p10x^3 + 15631/35730480*p10x^2]

In [362]:
lprint(A*vector(sols))

$ \left(\frac{159171600659883}{939768911869600000} \, \mathit{p10x}^{3} - \frac{64003849}{63123848000} \, \mathit{p10x}^{2} + \frac{49}{1060} \, \mathit{p10x},\,\frac{2318328964638127}{939768911869600000} \, \mathit{p10x}^{3} - \frac{237572041}{63123848000} \, \mathit{p10x}^{2} + \frac{21}{1060} \, \mathit{p10x},\,\frac{342091783031817}{939768911869600000} \, \mathit{p10x}^{3} - \frac{8113371}{63123848000} \, \mathit{p10x}^{2} + \frac{91}{1060} \, \mathit{p10x},\,\frac{2715298924835889}{939768911869600000} \, \mathit{p10x}^{3} - \frac{250990887}{63123848000} \, \mathit{p10x}^{2} + \frac{7}{1060} \, \mathit{p10x},\,-\frac{204317910793059}{234942227967400000} \, \mathit{p10x}^{3} + \frac{12073747}{15780962000} \, \mathit{p10x}^{2} + \frac{28}{265} \, \mathit{p10x},\,-\frac{22451394916951}{234942227967400000} \, \mathit{p10x}^{3} + \frac{7755083}{15780962000} \, \mathit{p10x}^{2} + \frac{42}{265} \, \mathit{p10x}\right) $

In [376]:
fsols = list(A*vector(sols)(p10x = 0.1))
fsols

[0.00461267147391930,
 0.00194596312769541,
 0.00858398436734415,
 0.000623505026450845,
 0.0105728189149292,
 0.0158538752441120]

In [377]:
#fsols = [random.random()/1000 for i in range(0,6)] # can check these are giving solutions

In [378]:
coords = { list(var_dict_old.keys())[i] : fsols[i] for i in [0,1,2,3,4,5] }
coords['l'] = 176/640
coords

{'p10x': 0.00461267147391930,
 'p10y': 0.00194596312769541,
 'p11x': 0.00858398436734415,
 'p11y': 0.000623505026450845,
 'p20x': 0.0105728189149292,
 'p21x': 0.0158538752441120,
 'l': 11/40}

In [379]:
funcs[0].function(*list(var_dict_old.values()))(**coords)

0.709418716631623*sqrt(2)*(1.41610680395338*sqrt(2) - 2) - 0.00268448707144790

In [380]:
sym_funcs = [func.function(*list(var_dict_old.values())) for func in funcs]

In [381]:
[float(func(**coords)) for func in sym_funcs]

[1.7149528768229012e-06,
 1.3814439892910306e-06,
 3.145432831639424e-06,
 1.5336612105556688e-06,
 5.6304162894017795e-06,
 3.793566979297478e-06]

In [383]:
lprint(sym_funcs[0])

$ \left( \mathit{p10x}, \mathit{p10y}, \mathit{p11x}, \mathit{p11y}, \mathit{p20x}, \mathit{p21x} \right) \ {\mapsto} \ \frac{\sqrt{2} {\left(\sqrt{2} \sqrt{{\left(\mathit{p10x} + 1\right)}^{2} + {\left(\mathit{p10y} - 1\right)}^{2}} - 2\right)} {\left(\mathit{p10x} + 1\right)}}{\sqrt{{\left(\mathit{p10x} + 1\right)}^{2} + {\left(\mathit{p10y} - 1\right)}^{2}}} + \frac{2 \, {\left(\mathit{p10x} - \mathit{p11x}\right)} {\left(\sqrt{{\left(\mathit{p10x} - \mathit{p11x}\right)}^{2} + {\left(\mathit{p10y} - \mathit{p11y} - 1\right)}^{2}} - 1\right)}}{\sqrt{{\left(\mathit{p10x} - \mathit{p11x}\right)}^{2} + {\left(\mathit{p10y} - \mathit{p11y} - 1\right)}^{2}}} + \frac{2 \, {\left(\mathit{p10x} - \mathit{p20x} - 1\right)} {\left(\sqrt{{\left(\mathit{p10x} - \mathit{p20x} - 1\right)}^{2} + \mathit{p10y}^{2}} - 1\right)}}{\sqrt{{\left(\mathit{p10x} - \mathit{p20x} - 1\right)}^{2} + \mathit{p10y}^{2}}} + \frac{2 \, {\left(\mathit{p10x} + 1\right)} {\left(\sqrt{{\left(\mathit{p10x} + 1\right)}^{2} + \mathit{p10y}^{2}} - 1\right)}}{\sqrt{{\left(\mathit{p10x} + 1\right)}^{2} + \mathit{p10y}^{2}}} $